<a href="https://colab.research.google.com/github/deitar/JiraWiki2Docx/blob/main/JiraWiki2Docx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install xmltodict
!pip install htmldocx
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184491 sha256=adbd33b981cf24676474870fec9d8223f85aaf79497c7d3e0340b47465f048c4
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [3]:
import os
import re
import html
import json
import xmltodict
import requests
from datetime import datetime
from requests.auth import HTTPBasicAuth
from docx import Document
from htmldocx import HtmlToDocx
from docx.shared import Inches
from docx.enum.table import WD_TABLE_ALIGNMENT, WD_ALIGN_VERTICAL
from docx.shared import Inches, RGBColor, Pt, Cm
from docx.enum.text import WD_LINE_SPACING
from docx import Document
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from itertools import groupby
from datetime import datetime
import numpy as np
import pandas as pd
from enum import Enum

In [4]:
sample_jira_text = '''h2. -Carried- out an +analysis+ to _identify_ {{customers}} who carried out POS/WEB (card) transactions during the period of cash scarcity but subsequently stopped transacting on this channel and map out the transactional behaviour of these customers prior to and during cash scarcity.

*Definition of Terms*

* Cash scarcity period: November 2022 to March 2023 (150 days)
** 6 months prior to cash scarcity: May 2022 to October 2022 (180 days)
* Period after cash scarcity: April 2023 to May 2023 (60 days)
**** Period after cash scarcity: April 2023 to May 2023 (60 days)

# test number 1
#* test number 11
#* test number 11
# test number 2
## test number 2

*Key Highlights:*

* Customers transactional behaviour before, during and after cash scarcity

||Period||Transacted before, during {color:#ff5630}AND {color}after cash scarcity||Transacted during cash scarcity {color:#ff5630}BUT {color}stopped after cash scarcity||{color:#ff5630}STARTED {color}transacting during cash scarcity {color:#ff5630}AND {color}continued afterwards||
|Distinct Card Holders|10,674,893|{color:#ff5630}*310,454*{color}|1,002,235|

||* go
* be||
| |
| |


* *1.6 million (66%)* of the customers who completed card transactions during the cash scarcity period also carried out transactions prior to and after the cash scarcity period.
* *13% (1 million)* of the customers who carried out POS/WEB transactions during cash scarcity stopped card transactions at the end of the cash scarcity period
* Customers with POS/WEB transactions during cash scarcity but not after cash scarcity

||Period||Transacted prior to AND during cash scarcity BUT subsequently stopped||Transacted only during cash scarcity, NOT before or after cash scarcity||% of customers who transacted only during cash scarcity||
|Distinct Card Holders|{color:#36b37e}30,063 {color}|{color:#ff991f}20,391{color}|{color:#ff991f}41%{color}|

* *29% (3.03 of 11 million)* of these customers also carried out POS/WEB transactions in the 6 months prior to cash scarcity which means that they regularly transacted using their cards regardless of cash scarcity.-'''


In [5]:
class JiraWikiBlockTypes(Enum):
    LIST = 'list'
    HEADING = 'heading'
    TABLE = 'table'
    CODE = 'code'
    QUOTE = 'quote'
    PARAGRAPH = 'paragraph'

In [6]:
def detect_jira_block_type(jira_block):
    jira_block_trimmed = jira_block.strip()
    marker_heading = r'^h([1-6])\..*$'
    # marker_list = '^([\*|\-|\#]+)\s(.*)'
    marker_list = '^[*#-]+\s.*$'
    marker_table = '^[||]+.*$'
    if re.match(marker_heading, jira_block_trimmed, re.DOTALL):
        return JiraWikiBlockTypes.HEADING
    if re.match(marker_list, jira_block_trimmed, re.DOTALL):
        return JiraWikiBlockTypes.LIST
    if re.match(marker_table, jira_block_trimmed, re.DOTALL):
        return JiraWikiBlockTypes.TABLE
    return JiraWikiBlockTypes.PARAGRAPH

In [7]:
def get_jira_blocks(jira_text):
    return re.split('\n\n', jira_text)

In [8]:
def write_heading_jira_to_doc(text, doc_part_to_write_to):
    pattern_heading = r'^h([1-6])\.(.*)'
    heading_match = re.match(pattern_heading, text)
    heading_number, heading_text = heading_match.groups()
    para = doc_part_to_write_to.add_paragraph()
    para.style = f'Heading {heading_number}'
    para.add_run(text=heading_text)

In [9]:
def write_list_jira_to_doc(text, doc_part_to_write_to):
    pattern_list_item = r'^([\*|\-|\#]+)\s{1}(.*)'
    # split block into lines
    lines = re.split('\n', text)
    for line in lines:
        list_type, list_text = re.match(pattern_list_item, line).groups()
        # issue 1: couldn't resolve the round and square bullets differently
        # issue 2: find a way of decorating nested lists with different point styles
        # issue 3: sublevel list have unwanted top space to their parent list
        max_list_level = 3
        style_name = "List Number" if list_type[:max_list_level][-1] == '#' else 'List Bullet'
        style_name = f'{style_name} {str(min(len(list_type), max_list_level)) if len(list_type) > 1 else ""}'.strip()
        para = doc_part_to_write_to.add_paragraph()
        para.style = style_name
        para.add_run(text = list_text)

In [10]:
def write_generic_jira_to_doc(text, doc_part_to_write_to):
    para = doc_part_to_write_to.add_paragraph()
    para.add_run(text = text)

In [11]:
def write_jira_block_to_doc(jira_block):

    # handle leading new lines
    new_lines, actual_jira_block = re.match('(\n*)(.*)', jira_block, re.DOTALL).groups()
    num_new_lines = new_lines.count('\n')

    # add new lines by occurence
    for i in range(num_new_lines):
        para = doc.add_paragraph()
        para.add_run()

    # 1. heading
    if detect_jira_block_type(actual_jira_block) == JiraWikiBlockTypes.HEADING:
        write_heading_jira_to_doc(actual_jira_block, doc)
    # 2. list
    elif detect_jira_block_type(actual_jira_block) == JiraWikiBlockTypes.LIST:
        write_list_jira_to_doc(actual_jira_block, doc)
    # 3. table
    elif detect_jira_block_type(actual_jira_block) == JiraWikiBlockTypes.TABLE:
        write_generic_jira_to_doc(actual_jira_block, doc)

    # any other
    else:
        write_generic_jira_to_doc(actual_jira_block, doc)

        # handle text effects



In [11]:
# Initialize a new Document
doc = Document()

In [12]:
for block in get_jira_blocks(sample_jira_text):
    print(repr(block))
    write_jira_block_to_doc(block)

'h2. -Carried- out an +analysis+ to _identify_ {{customers}} who carried out POS/WEB (card) transactions during the period of cash scarcity but subsequently stopped transacting on this channel and map out the transactional behaviour of these customers prior to and during cash scarcity.'
'*Definition of Terms*'
'* Cash scarcity period: November 2022 to March 2023 (150 days)\n** 6 months prior to cash scarcity: May 2022 to October 2022 (180 days)\n* Period after cash scarcity: April 2023 to May 2023 (60 days)\n**** Period after cash scarcity: April 2023 to May 2023 (60 days)'
'# test number 1\n#* test number 11\n#* test number 11\n# test number 2\n## test number 2'
'*Key Highlights:*'
'* Customers transactional behaviour before, during and after cash scarcity'
'||Period||Transacted before, during {color:#ff5630}AND {color}after cash scarcity||Transacted during cash scarcity {color:#ff5630}BUT {color}stopped after cash scarcity||{color:#ff5630}STARTED {color}transacting during cash scarci

In [13]:
doc.save('output.docx')

In [98]:
# test_block = '\n\n* *3.6 million (66%)* of the +to+ and after the cash scarcity period.\n* *23% (1.31 million)* ed out POS/WEB transactions d'
test_block = '_3.6_ {{mil_lion_ (66%)}} of the to *and* after _the_ {color:green}cash scarcity{color} period.\n'
# text_strong_matches = re.findall('\*\S[^*]*\S\*', test_block, re.DOTALL)
# print(text_strong_matches)
pattern_strong_effect = '((\*)(\S[^*]*\S)(\*))'
pattern_italics_effect = '((\_)(\S[^_]*\S)(\_))'
pattern_color_effect = '((\{color:[A-Za-z]*\})(\S((?!{color}).)*\S)(\{color\}))'
pattern_text_effect = re.compile(f'{pattern_strong_effect}|{pattern_italics_effect}|{pattern_color_effect}', re.IGNORECASE | re.DOTALL)
for m in pattern_text_effect.finditer(test_block):
    # print(m.start(), '\t', m.groups(), '\t', m.end(), '\t') # test_block[m.start()])
    print(list(filter(lambda x: x, m.groups())))

['_3.6_', '_', '3.6', '_']
['_lion_', '_', 'lion', '_']
['*and*', '*', 'and', '*']
['_the_', '_', 'the', '_']
['{color:green}cash scarcity{color}', '{color:green}', 'cash scarcity', 't', '{color}']


In [79]:
test_block = '_3.6_ *mil_lion_ (66%)* of the to *and* after _the_ cash scarcity period.\n'

text_effect_para_dict = {}

def effecter(text, existing_effect=None):
  global text_effect_para_dict
  # asterisk/markup with no space after start or before end
  pattern_strong_effect = '((\*)(\S[^*]*\S)(\*))'
  pattern_italics_effect = '((\_)(\S[^_]*\S)(\_))'
  pattern_color_effect = '((\{color:[A-Za-z]*\})(\S((?!{color}).)*\S)(\{color\}))'
  print(text)
  pattern_text_effect = re.compile(f'{pattern_strong_effect}|{pattern_italics_effect}|{pattern_color_effect}', re.DOTALL|re.IGNORECASE)
  # if not pattern_text_effect.match(text):
  #     return
  for m in pattern_text_effect.finditer(text):
      full_match, text_effect_ope_tag, inner_text, text_effect_cls_tag = tuple(filter(lambda x: x, m.groups()))
      # print(full_match, '\t', text_effect_ope_tag, '\t', inner_text, '\t', text_effect_cls_tag)
      # text_effect_markup = test_block[m.start()]
      # inner_text = m.group().strip(text_effect_markup)
      text_effect_para_dict[m.start()] = inner_text
      effecter(inner_text)

effecter(test_block)

_3.6_ *mil_lion_ (66%)* of the to *and* after _the_ cash scarcity period.

3.6
mil_lion_ (66%)
lion
and
the
